In [1]:
import pandas as pd


In [2]:
def delete_duplicates_and_save(input_file, inputs_duplicates, inputs_na):
    df = pd.read_csv(input_file)
    df_unique = df.drop_duplicates(subset=inputs_duplicates)
    df_unique = df_unique.dropna(subset=inputs_na)
    df_unique.to_csv(input_file, index=False)

In [3]:
# delete_duplicates_and_save("SteamAppsList.csv",["appid", "name"], ["appid", "name"])

In [4]:
# delete_duplicates_and_save("SteamGamesMetadata.csv",["appid", "name"], ["appid", "name"])

In [5]:
# delete_duplicates_and_save("Clean_SteamHistory_PCout.csv", ["appid", "name"], ["appid", "name", "date_playerscount"])

In [28]:
df = pd.read_csv("Clean_SteamHistory_PCout.csv")
df.info()
df = df.drop_duplicates(subset=["name", "appid"])
df.info()
df = df.dropna(subset=["date_playerscount","name"])
df.info()
df.count()
# df.to_csv("Clean_SteamHistory_PCout.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7543 entries, 0 to 7542
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              7543 non-null   int64 
 1   name               7543 non-null   object
 2   header_image       7543 non-null   object
 3   short_description  7464 non-null   object
 4   developers         7512 non-null   object
 5   publishers         7492 non-null   object
 6   release_date       7522 non-null   object
 7   platforms          7543 non-null   object
 8   price              5832 non-null   object
 9   categories         7436 non-null   object
 10  genres             7373 non-null   object
 11  website            5269 non-null   object
 12  screenshots        7414 non-null   object
 13  background         7543 non-null   object
dtypes: int64(1), object(13)
memory usage: 825.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7543 entries, 0 to 7542
Data columns (

KeyError: ['date_playerscount']

In [22]:
import time
df = pd.read_csv("Clean_SteamHistory_PCout.csv")
games_that_dont_have_players = []
currnet_time = int(time.time() * 1000)
one_day_ago = currnet_time - 86400000
one_week_ago = currnet_time - 604800000

for data in df.iterrows():
    split_to_last = data[1]["date_playerscount"].split(", ")[-1]
    current_data_split = split_to_last.split(" ")
    timestamp_ms = int(current_data_split[0])
    if not (one_week_ago <= timestamp_ms <= currnet_time):
        games_that_dont_have_players.append(data[1]["appid"])
        continue

    all_players = []
    for current_data in data[1]["date_playerscount"].split(", "):
        current_data_split = current_data.split(" ")
        players_count = int(current_data_split[1])
        all_players.append(players_count)

    if max(all_players) <= 3:
        games_that_dont_have_players.append(data[1]["appid"])
    if  int(split_to_last.split(" ")[1]) <= 3:
        games_that_dont_have_players.append(data[1]["appid"])


In [ ]:
df = df[~df["appid"].isin(games_that_dont_have_players)]
df.info()
# df.to_csv("Clean_SteamHistory_PCout.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 7543 entries, 0 to 8505
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              7543 non-null   int64 
 1   name               7543 non-null   object
 2   header_image       7543 non-null   object
 3   short_description  7464 non-null   object
 4   developers         7512 non-null   object
 5   publishers         7492 non-null   object
 6   release_date       7522 non-null   object
 7   platforms          7543 non-null   object
 8   price              5832 non-null   object
 9   categories         7436 non-null   object
 10  genres             7373 non-null   object
 11  website            5269 non-null   object
 12  screenshots        7414 non-null   object
 13  background         7543 non-null   object
dtypes: int64(1), object(13)
memory usage: 883.9+ KB


In [ ]:
df = pd.read_csv("SteamGamesMetadata.csv")
df2 = pd.read_csv("Clean_SteamHistory_PCout.csv")
df = df[df["appid"].isin(df2["appid"])]
df.info()
# df.to_csv("SteamGamesMetadata.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 7543 entries, 0 to 8505
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              7543 non-null   int64 
 1   name               7543 non-null   object
 2   header_image       7543 non-null   object
 3   short_description  7464 non-null   object
 4   developers         7512 non-null   object
 5   publishers         7492 non-null   object
 6   release_date       7522 non-null   object
 7   platforms          7543 non-null   object
 8   price              5832 non-null   object
 9   categories         7436 non-null   object
 10  genres             7373 non-null   object
 11  website            5269 non-null   object
 12  screenshots        7414 non-null   object
 13  background         7543 non-null   object
dtypes: int64(1), object(13)
memory usage: 883.9+ KB


In [ ]:
bad_apis_txt = "bad_fetching_appids.txt"
bad_apis = []
if bad_apis_txt:
    with open(bad_apis_txt, "r") as f:
        bad_apis = [line.strip() for line in f.readlines()]

df = pd.read_csv("SteamGamesMetadata.csv")
df2 = pd.read_csv("Clean_SteamHistory_PCout.csv")

df2 = df2[df2["appid"].isin(df["appid"])]
df2 = df2[~df2["appid"].isin(bad_apis)]

df2.info()
# df2.to_csv("Clean_SteamHistory_PCout.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7543 entries, 0 to 7542
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              7543 non-null   int64 
 1   name               7543 non-null   object
 2   header_image       7543 non-null   object
 3   short_description  7464 non-null   object
 4   developers         7512 non-null   object
 5   publishers         7492 non-null   object
 6   release_date       7522 non-null   object
 7   platforms          7543 non-null   object
 8   price              5832 non-null   object
 9   categories         7436 non-null   object
 10  genres             7373 non-null   object
 11  website            5269 non-null   object
 12  screenshots        7414 non-null   object
 13  background         7543 non-null   object
dtypes: int64(1), object(13)
memory usage: 825.1+ KB
